In [1]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()

## IMPORTS and SETUP

In [2]:
# general imports
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import networkx as nx
# import seaborn as sns
import time
from datetime import datetime
import pickle
import random
from braket.aws import AwsDevice
# magic line for producing visualizations in notebook
%matplotlib inline

In [3]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(a=seed)

# switch to trigger writing training results to disk
store_results = False
# switch to trigger printing results of each optimization cycle
verbose = True

In [4]:
# AWS imports: Import Braket SDK modules
from braket.circuits import FreeParameter
from braket.devices import LocalSimulator, Devices

In [5]:
from utils_classical import *

In [6]:
# Set up device: Local Simulator
# device = LocalSimulator()

## In this notebook I will run the problem on real hardware.

In [7]:
## example code for other backends
# from braket.aws import AwsDevice
# from braket.devices import Devices
## choose the on-demand simulator to run your circuit
# device = AwsDevice(Devices.Amazon.SV1)
## choose the Rigetti device to run your circuit
# device = AwsDevice(Devices.Rigetti.Ankaa2)
## choose the Ionq device to run your circuit
device = AwsDevice(Devices.IonQ.Aria1)
## choose the IQM device to run your circuit
# device = AwsDevice(Devices.IQM.Garnet)

## PROBLEM SETUP

We consider a graph coloring problem.
Given a graph $G=(V,E)$, made of a set vertices (also called nodes) $V$ and edges $E$, our goal is to color each node red or blue, then score a point for each node that is next to a node of different color. 
We strive to find the optimal coloring that scores the largest number of points.
To this end, we will address the dual problem of finding the minimum energy of the corresponding Ising Hamiltonian. 
To get started, we first use the open-source ```networkx``` library to visualize the problem graph. 
Feel free to play with the parameters $n$ (for the number of nodes) and $m$ (for the number of edges) below to consider other graphs of your choice. 

In [ ]:
# # setup Erdos Renyi graph
# n = 10  # number of nodes/vertices
# m = 20  # number of edges

# # define graph object
# G = nx.gnm_random_graph(n, m, seed=seed)
# # G = nx.erdos_renyi_graph(n=n, p=0.6, seed=seed)
# # positions for all nodes
# pos = nx.spring_layout(G)

# # choose random weights
# for (u, v) in G.edges():
#     G.edges[u,v]['weight'] = np.round(random.uniform(0, 1),4)

# # draw graph
# nx.draw(G, pos, with_labels = True, font_weight = 'bold')
# labels = nx.get_edge_attributes(G, 'weight')
# nx.draw_networkx_edge_labels(G, pos, edge_labels = labels)
# plt.show()

edges = [(0,1,1),(2,3,1),(3,0,1), (2,0,1), (0,3,1)]
nodes = []
for edge in edges:
    start_node = edge[0]
    end_node = edge[1]
    if (start_node not in nodes):
        nodes.append(start_node)
    if (end_node not in nodes):
        nodes.append(end_node)
num_nodes = len(nodes)
G = nx.Graph() 
G.add_nodes_from(nodes)
G.add_weighted_edges_from(edges)

pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels = True, font_weight = 'bold')
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels = labels)

In [9]:
# set Ising matrix 
Jfull = nx.to_numpy_array(G)

# get off-diagonal upper triangular matrix
J = np.triu(Jfull, k=1).astype(np.float64)

In [10]:
# plot Ising matrix
# plt.figure(1, figsize=[7, 5])
# sns.heatmap(J, annot=True,  linewidths=.5, cmap="YlGnBu", annot_kws = {'alpha': 1})
# plt.title('Ising distance matrix');
# plt.tight_layout();

## IMPLEMENTATION OF QAOA WITH BRAKET 

In this section we load a set of useful helper functions that we will explain in detail below. 
Specifically in ```utils_qaoa.py``` we provide simple building blocks for the core modules of our QAOA algorithm, that is (i) a function called ```circuit``` that defines the parametrized ansatz, (ii) a function called ```objective_function``` that takes a list of variational parameters as input, and returns the cost associated with those parameters and finally (iii) a function ```train``` to run the entire QAOA algorithm for given ansatz. 
This way we can solve the problem in a clean and modular approach.
Here, we show in markdown the definition of the parametrized QAOA circuit. 
For more details, see the corresponding file ```utils_qaoa.py```. 

In [11]:
from utils_qaoa import *

```python
# function to implement evolution with driver Hamiltonian
def driver(beta, n_qubits):
    """
    Returns circuit for driver Hamiltonian U(Hb, beta)
    """
    # instantiate circuit object
    circ = Circuit()
    
    for qubit in range(n_qubits):
        gate = Circuit().rx(qubit, 2*beta)
        circ.add(gate)
    
    return circ


# helper function for evolution with cost Hamiltonian
def cost_circuit(gamma, n_qubits, ising):
    """
    returns circuit for evolution with cost Hamiltonian
    """
    # instantiate circuit object
    circ = Circuit()

    # get all non-zero entries (edges) from Ising matrix 
    idx = ising.nonzero()
    edges = list(zip(idx[0], idx[1]))
    
    for qubit_pair in edges:
        # get interaction strength
        int_strength = ising[qubit_pair[0], qubit_pair[1]]
        # for Rigetti we decompose ZZ using CNOT gates
        if 'Aspen' in device.name:
            gate = ZZgate(qubit_pair[0], qubit_pair[1], gamma*int_strength)
            circ.add(gate)
        # classical simulators and IonQ support ZZ gate
        else:
            gate = Circuit().zz(qubit_pair[0], qubit_pair[1], angle=2*gamma*int_strength)
            circ.add(gate)

    return circ


# function to build the QAOA circuit with depth p
def circuit(params, n_qubits, ising):
    """
    function to return full QAOA circuit
    """

    # initialize qaoa circuit with first Hadamard layer: for minimization start in |->
    circ = Circuit()
    X_on_all = Circuit().x(range(0, n_qubits))
    circ.add(X_on_all)
    H_on_all = Circuit().h(range(0, n_qubits))
    circ.add(H_on_all)

    # setup two parameter families
    circuit_length = int(len(params) / 2)
    gammas = params[:circuit_length]
    betas = params[circuit_length:]

    # add circuit layers
    for mm in range(circuit_length):
        circ.add(cost_circuit(gammas[mm], n_qubits, ising))
        circ.add(driver(betas[mm], n_qubits))

    return circ

```

## VISUALIZATION OF THE QAOA ANSATZ

Let us first visualize our parametrized QAOA ansatz for a small number of qubits and fixed (i.e., not optimized) parameters. 
For convenience, the parameters are displayed in the circuit (up to a factor of $2$ we have added in our ansatz definition). 
First we prepare the state $|0,0,\dots\rangle \rightarrow |-,-,\dots\rangle$, with the superposition state $|-\rangle = (|0\rangle -|1\rangle  )/\sqrt{2}$. 
Following the discussion above, we choose to start out with this state as it is the minimal energy state of the simple driver Hamiltonian $H_{B}$. 
This state preparation is followed by one layer of the QAOA ansatz, consisting of evolution with the cost Hamiltonian by $\exp(-i\gamma H_{C})= \prod_{j,l}\exp(-i\gamma J_{j,l}\sigma_{j}^{z}\sigma_{l}^{z}) = \prod_{j,l} ZZ_{j,l}(2\gamma J_{j,l})$, followed by the single-qubit driving term, $\exp(-i\beta H_{B})= \prod_{j} \exp(-i\beta \sigma_{j}^{x})= \prod_{j} R_{j}^{(x)}(2\beta)$.
Note that the circuit definition depends on the ```device``` object, as the implementation of the ZZ gate depends on the specific gate set supported by the device.  

In [ ]:
# create parameters
gammas = [FreeParameter('gamma')]
betas = [FreeParameter('beta')]
params = gammas + betas

# for demonstration purposes use small Ising matrix
J_sub = np.array([[0, 1], [0, 0]])
N = J_sub.shape[0]

# get circuit ansatz
my_simple_circuit = circuit(params, device, N, J_sub)

# print test ansatz circuit
print('Printing test circuit:')
print(my_simple_circuit)

We see that our ansatz produces the expected result for shallow QAOA with $p=1$. 
We run one more sanity check for $p=2$ below. 

In [ ]:
# set number of qubits and fix parameters
gammas = [FreeParameter('gamma_1'), FreeParameter('gamma_2')]
betas = [FreeParameter('beta_1'), FreeParameter('beta_2')]
params = gammas + betas

# for demonstration purposes use small Ising matrix
J_sub = np.array([[0, 1], [0, 0]])
N = J_sub.shape[0]

# get circuit ansatz
my_simple_circuit = circuit(params, device, N, J_sub)

# print test ansatz circuit
print('Printing test circuit:')
print(my_simple_circuit)

## QAOA SIMULATION ON LOCAL SCHROEDINGER SIMULATOR

We are now all set to run some QAOA simulation experiments. 
First of all, you can play and experiment yourself with the number of qubits $N$. 
Secondly, you may also experiment with the classical optimizer. 
Since we are using an off-the-shelf, black-box ```scipy``` minimizer (as described in more detail [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)), you can simply swap between different optimizers by setting the ```OPT_METHOD``` parameter below. 
Some popular options readily available within this library include *Nelder-Mead*, *BFGS* and *COBYLA*. 
As a precautionary warning, note that the classical optimization step may get stuck in a local optimum, rather than finding the global minimum for our parametrized QAOA ansatz wavefunction. 
To address this issue, we may run several optimization loops, starting from different random parameter seeds. 
While this brute-force approach does not provide any guarantee to find the global optimum, from a pragmatic point of view at least it does increase the odds of finding an acceptable solution, at the expense of potentially having to run many more circuits on the simulator or QPU, respectively.
Finally, the optimization loop may require the execution of many individual quantum tasks (i.e., single circuit executions for fixed parameters). 
For example, when choosing the classical [Powell](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-powell.html#optimize-minimize-powell) optimizer for the graph considered here, we find $\sim 270$ cycles in the for loop. 
For the local simulator device chosen here by default this is not an issue, but if you run this algorithm on any QPU you may want to adjust the ```maxfev``` parameter to control the maximum allowed number function evaluations (compare comment in the next code block below).

In [14]:
##################################################################################
# set up hyperparameters
##################################################################################

# User-defined hypers
DEPTH = 12  # circuit depth for QAOA
SHOTS = 100  # number measurements to make on circuit
OPT_METHOD = 'COBYLA'  # SLSQP, COBYLA, Nelder-Mead, BFGS, Powell, ...

# set up the problem
n_qubits = J.shape[0]

# initialize reference solution (simple guess)
bitstring_init = -1 * np.ones([n_qubits])
energy_init = np.dot(bitstring_init, np.dot(J, bitstring_init))

# set tracker to keep track of results
tracker = {
    'count': 1,                           # Elapsed optimization steps
    'optimal_energy': energy_init,        # Global optimal energy
    'opt_energies': [],                   # Optimal energy at each step
    'global_energies': [],                # Global optimal energy at each step
    'optimal_bitstring': bitstring_init,  # Global optimal bitstring
    'opt_bitstrings': [],                 # Optimal bitstring at each step
    'costs': [],                          # Cost (average energy) at each step
    'res': None,                          # Quantum result object
    'params': [],                         # Track parameters
    'ratio': [],                          # The benchmarking metric
    'ratio_min': [],                      # Best possible solution during measurement
    'ratio_shot': [],                     # The ratio for the optimal solution over all measurements
    'prob': []                            # Record prob of measurement
}

# set options for classical optimization
options = {'disp': True, 'maxiter': 10}

In [ ]:
##################################################################################
# run QAOA optimization on graph 
##################################################################################

print('Circuit depth hyperparameter:', DEPTH)
print('Problem size:', n_qubits)

# kick off training
start = time.time()
result_energy, result_angle, tracker = train(
    device = device, options=options, p=DEPTH, ising=J, n_qubits=n_qubits, n_shots=SHOTS, 
    opt_method=OPT_METHOD, tracker=tracker, verbose=verbose, G=G)
end = time.time()

# print execution time
print('Code execution time [sec]:', end - start)

# print optimized results
print('Optimal energy:', tracker['optimal_energy'])
print('Optimal classical bitstring:', tracker['optimal_bitstring'])

##################################################################################
# Compute output and dump to pickle
##################################################################################

if store_results:
    out = {'p': DEPTH, 'N': n_qubits,
           'ENERGY_OPTIMAL': tracker['optimal_energy'], 'BITSTRING': tracker['optimal_bitstring'],
           'result_energy': result_energy, 'result_angle': result_angle}

    # store results: dump output to pickle with timestamp in filename
    time_now = datetime.strftime(datetime.now(), '%Y%m%d%H%M%S')
    results_file = 'results-'+time_now+'.pkl'
    print(f'Writing results to file: {results_file}')
    pickle.dump(out, open(results_file, "wb"))
    
    # you can load results as follows
    # out = pickle.load(open(results_file, "rb"))

## POSTPROCESSING AND COMPARISON OF OUR QAOA RESULTS WITH CLASSICAL RESULTS

without error_mitigation

In [ ]:
import pandas as pd

debias_off = pd.DataFrame.from_dict(tracker['prob'], orient="index").rename(columns={0: "debias off"})

In [ ]:
# visualize the optimization process
cycles = np.arange(1, tracker['count'])

In [ ]:
# # get results for variational angles
# gamma = result_angle[:DEPTH]
# beta = result_angle[DEPTH:]
# # get array [1, 2, ..., p]
# pa = np.arange(1, DEPTH + 1)

# plt.figure(2)
# plt.plot(pa, gamma / np.pi, '-o', label='gamma')
# plt.plot(pa, beta / np.pi, '-s', label='beta')
# plt.xlabel('circuit depth (layer) p')
# plt.ylabel('optimal angles [pi]')
# plt.xticks(pa)
# plt.legend(title='Variational QAOA angles:', loc='upper left')
# plt.show()

In [ ]:
# visualize solution
colorlist = tracker['optimal_bitstring']
print('Optimal cut found with QAOA:', tracker['optimal_bitstring'])
colorlist[colorlist == -1] = 0
# plot_colored_graph(J, N, colorlist, pos)
plot_colored_graph_simple(G, colorlist, pos)
print('Minimal energy found with QAOA:', tracker['optimal_energy'])

In [ ]:
plt.xlabel('optimization cycle')
plt.ylabel('best minimum energy at each step')
plt.plot(cycles, tracker['ratio'], label='expected')
plt.plot(cycles, tracker['ratio_min'], label='optimal')
plt.legend()
plt.show()

In [ ]:
plt.xlabel('optimization cycle')
plt.ylabel('percentage for the optimal solution')
plt.plot(cycles, tracker['ratio_shot'])
plt.show()

with error mitigation

In [ ]:
start = time.time()
result_energy, result_angle, tracker_em = train(
    device = device, options=options, p=DEPTH, ising=J, n_qubits=n_qubits, n_shots=SHOTS, 
    opt_method=OPT_METHOD, tracker=tracker, verbose=verbose, G=G, error_mitigation=True)

end = time.time()

# print execution time
print('Code execution time [sec]:', end - start)

# print optimized results
print('Optimal energy:', tracker['optimal_energy'])
print('Optimal classical bitstring:', tracker['optimal_bitstring'])

In [ ]:
sharpen_on = pd.DataFrame.from_dict(tracker_em['prob'], orient="index").rename(
    columns={0: "debias on, sharpened"}
)
df = debias_off.join(sharpen_on)
df2 = df.sort_values(by="debias off").tail(10)
df2

In [ ]:
# visualize solution
colorlist = tracker['optimal_bitstring']
print('Optimal cut found with QAOA:', tracker_em['optimal_bitstring'])
colorlist[colorlist == -1] = 0
# plot_colored_graph(J, N, colorlist, pos)
plot_colored_graph_simple(G, colorlist, pos)
print('Minimal energy found with QAOA:', tracker_em['optimal_energy'])

In [ ]:
plt.xlabel('optimization cycle')
plt.ylabel('best minimum energy at each step')
plt.plot(cycles, tracker_em['ratio'], label='expected')
plt.plot(cycles, tracker_em['ratio_min'], label='optimal')
plt.legend()
plt.show()

In [ ]:
plt.xlabel('optimization cycle')
plt.ylabel('percentage for the optimal solution')
plt.plot(cycles, tracker_em['ratio_shot'])
plt.show()